In [1]:
import csv
import numpy as np
import tensorflow as tf

from itertools import islice

In [2]:
def load_data(file_name = 'train.csv' , one_hot = True):
    csv_reader = csv.reader(open(file_name))

    train_set = []
    count = 0
    train_labels = []
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row)) #类型转换
        train_set.append(row)

        temp = [0.,0.,0.,0.,0.,0.,0.,0.,0.,0.]
        temp[int(train_set[count][0])] = 1.
        train_labels.append(temp)

        count += 1
    
    for i in range(count):
        train_set[i] = train_set[i][1:785]
    
    train_set = np.matrix(train_set)
    train_labels = np.matrix(train_labels)
    
    return train_set , train_labels

In [3]:
#加载数据
train_set , train_labels = load_data()

In [4]:
train_set = train_set / 255.0
#print(train_set)
#print(train_labels)

In [5]:
#cnn add

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1 , shape=shape)
    return tf.Variable(initial)

def conv2d(x , W):
    return tf.nn.conv2d(x , W , strides=[1,1,1,1] , padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x , ksize=[1,2,2,1] , strides=[1,2,2,1] , padding='SAME')

#数据重构
xs = tf.placeholder(tf.float32 , [None , 784]) #28*28
ys = tf.placeholder(tf.float32 , [None , 10])
keep_prob = tf.placeholder(tf.float32)

#-1就是一次batch的size
x_image = tf.reshape(xs , [-1 , 28 , 28 , 1]) #28*28的图片 通道为1（只有黑白色彩） 彩色就3 （RGB）

#========
#conv1
#首先是卷积核
#长 宽 通道数量 卷积核个数（feature map个数）
#一个feature map使用一个weight和一个bias
W_conv1 = weight_variable([5 , 5 , 1 , 32])
#每一个feature map共用一个bias
#32个 feature map 所以使用32个bias
b_conv1 = bias_variable([32])
#relu激活

#padding还有VALID方式
h_conv1 = tf.nn.relu(conv2d(x_image , W_conv1) + b_conv1)#28*28*32 因为padding 为‘SAME’ 所以图像尺寸没有发生变化 输出为32个feature map
#max池化
h_pool1 = max_pool_2x2(h_conv1)#14*14*32 因为pooling的size是2*2 故尺寸缩小为原来的1/2 还是32个feature map

#========
#conv2
W_conv2 = weight_variable([5,5,32,64])#第二个卷积层 产生64个feature map
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1 , W_conv2) + b_conv2)#14*14*64
h_pool2 = max_pool_2x2(h_conv2)#7*7*64

#========
#fully connect1
#将卷积层输出的图像信息变回2维
h_pool2_flat = tf.reshape(h_pool2 , [-1 , 7*7*64]) #-1表示不知道多少个样本 python自动计算 列数为7*7*64 一行一个样本
W_fc1 = weight_variable([7*7*64 , 1024])#全连接 为1024神经元
b_fc1 = bias_variable([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat , W_fc1) + b_fc1)
h_fc1_drop = tf.nn.dropout(h_fc1 , keep_prob)#防止overfitting

#========
#fully connect2
W_fc2 = weight_variable([1024 , 10])
b_fc2 = bias_variable([10])
prediction = tf.matmul(h_fc1_drop , W_fc2) + b_fc2

In [6]:
mini_batch = 0

def next_batch(train_set , train_labels , batch_size = 50):
    global mini_batch
    if mini_batch + batch_size > 42000:
        mini_batch = 0

    train_set_batch = train_set[mini_batch : mini_batch + batch_size]
    train_labels_batch = train_labels[mini_batch : mini_batch + batch_size]
    
    mini_batch = mini_batch + batch_size
    
    return train_set_batch , train_labels_batch

In [17]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction , labels=ys))
#-tf.reduce_sum(ys * tf.log(prediction + 1e-10))

train_step = tf.train.AdamOptimizer(0.01).minimize(loss)

sess= tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(100):
    train_set_batch , train_labels_batch = next_batch(train_set , train_labels)
    sess.run(train_step , feed_dict = {xs:train_set_batch , ys:train_labels_batch , keep_prob:0.8})
    print(sess.run(loss , feed_dict = {xs:train_set_batch , ys:train_labels_batch , keep_prob:0.8}))

66.3579
46.4316
7.10662
2.63844
2.52945
2.6148
2.36937
2.63521
2.52319
2.35347
2.18342
1.92692
2.10963
1.67048
1.82099
1.48772
1.6829
1.33412
1.16525
1.0439
1.04156
1.19907
1.08739
0.951698
0.808418
1.01813
0.770298
0.787483
0.655927
0.524133
0.548318
0.857671
0.656665
0.489155
0.379253
0.628915
0.863418
0.533263
0.605751
0.670518
0.475306
0.229157
0.455454
0.713327
0.229772
0.549607
0.261112
0.399788
0.393649
0.527245
0.364692
0.269676
0.554575
0.347891
0.378569
0.251422
0.406348
0.309332
0.431848
0.559792
0.623767
0.275286
0.181219
0.338481
0.433795
0.108607
0.159762
0.277572
0.412243
0.377445
0.302704
0.223652
0.2424
0.296083
0.131012
0.119845
0.0757381
0.363613
0.189785
0.296205
0.248628
0.275451
0.260419
0.0997222
0.185169
0.187688
0.188557
0.170859
0.320347
0.176334
0.184412
0.432193
0.165413
0.0491625
0.162401
0.15386
0.0694445
0.037214
0.270395
0.142799


In [ ]:
#dev-set
train_set_batch , train_labels_batch = next_batch(train_set , train_labels ,10000)

prediction_number = tf.argmax(tf.nn.softmax(prediction) , 1)
real_number = tf.argmax(ys , 1)
right = tf.equal(prediction_number , real_number)
accuracy = tf.reduce_mean(tf.cast(right , tf.float32))

print(sess.run(accuracy , feed_dict={xs:train_set_batch , ys:train_labels_batch , keep_prob:1.0} ))

#sess.close()

In [89]:
def load_test_set(file_name = 'test.csv'):
    csv_reader = csv.reader(open(file_name))
    test_set = []
    
    for row in islice(csv_reader ,1 , None):
        row = list(map(np.float32 , row))
        test_set.append(row)
        
    test_set = np.matrix(test_set)

    return test_set

In [131]:
test_set = load_test_set()
test_set = test_set / 255.0

In [206]:
test_set_labels = tf.argmax(prediction , 1)
result = sess.run(test_set_labels , feed_dict={x:test_set})

In [207]:
csv_file = open('sample_submission.csv' , 'w' , newline='')
writer = csv.writer(csv_file)
writer.writerow(['ImageId','Label'])

for i in range(28000):
    writer.writerow([i+1 , result[i]])

csv_file.close()